In [3]:
# importing required packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
from IPython.display import HTML


# navigating to CNN-US Stock web page
driver = webdriver.Chrome()
driver.get("https://money.cnn.com/data/us_markets/")

# loading page source info
time.sleep(5) # wait for 5 sec to load the web page
locate_page_xml = driver.find_element_by_xpath("//div[@id='wsod_usStocks']")
page_source_text = driver.page_source
soup = BeautifulSoup(page_source_text, "lxml")

find_stock_data = soup.find("div", class_="wsod_fRight")

# Data as of
data_as_of = find_stock_data.contents[1].text
# Closing Day
todays_day = find_stock_data.contents[3].text[:-1]

# load lxml data for stock section
raw_data = soup.find("div", id="wsod_tickerRoll").contents[5].find_all("li")

driver.quit()

# loop through the main U.S. Stocks and load data into a variable
records = []
for li in raw_data:
    symbol = li.find(class_="wsod_symbol").text
    price = li.find(class_="bannerQuote").find("span").text
    change = li.find(class_="quoteChange").text
    pct_change = li.find(class_="quotePctChange").text
    records.append((symbol, price, change, pct_change))

# load data into pandas dataframe
df = pd.DataFrame(records, columns= ["symbol", "price", "change", "pct_change"])

'''
Login into Email 
create email body with the stored variables texts 
send email
verify email was sent

'''
subject = "US Stock" + " " + todays_day + " " + "-" + " " + data_as_of
body = "https://money.cnn.com/data/us_markets/"
sender_email = email
receiver_email = email
port = port
password = password
context = ssl.create_default_context()

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject

# Add html body to email

html = """\
<html>
  <head></head>
  <body>
    {0}
  </body>
</html>
""".format(df.to_html(index=False))

partHTML = MIMEText(html, 'html')
message.attach(partHTML)

message.attach(MIMEText(body, "plain"))

text = message.as_string()

try:
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
      server.login("email", password)
      server.sendmail(sender_email, receiver_email, text)
      print('Sending email ...')
except:
    print('Something went wrong...')

Sending email ...
